In [ ]:
# Имя набор данных построить аппроксимационную модель
# Хьюбер, Тюки - функция потерь - это робастость
# Квадрат если небольшое
# Линия если больше
# Константа
# 2 - регуляризация L1, L2, SVD
# 3 - робастость

In [ ]:
# 4 Лаб работа - сплайны. Натуральный сплайн
# 4 Лаб работа - сплайны. Робастный сглаживающий сплайн

In [ ]:
!python -m pip install matplotlib numpy

In [ ]:
from sys import float_info

import numpy as np
from numpy.linalg import norm, solve

def ordinary_least_squares(x: np.ndarray, y: np.ndarray, order: int,
        p: int = 0, alpha: float = 1.0, eps: float = 1E-7, max_iter: int = 100):
    if (not isinstance(x, np.ndarray) or not isinstance(y, np.ndarray)
            or not isinstance(order, int) or not isinstance(p, int)
            or not isinstance(alpha, float) or not isinstance(eps, float)
            or not isinstance(max_iter, int)):
        raise TypeError()
    if (len(x.shape) != 1 or len(y.shape) != 1 or x.shape != y.shape
            or order < 2 or p not in range(3) or alpha < 0
            or eps < float_info.epsilon or max_iter < 1):
        raise ValueError()

    m, n = order + 1, len(x)
    x_upper = np.vander(x, m, increasing=False)
    left, right = x_upper.T @ x_upper, x_upper.T @ y
    print('Condition number =', np.linalg.cond(left))

    w_upper = alpha * np.eye(m)
    if p == 2:
        left += w_upper
        print('L2: Condition number =', np.linalg.cond(left))
    beta = solve(left, right)
    if p != 1:
        return beta

    for _ in range (max_iter):
        np.fill_diagonal(w_upper, 1.0 / np.abs(beta))
        next_beta = np.linalg.solve(left + alpha * w_upper, right)
        error = norm(next_beta - beta, ord='fro')
        beta, next_beta = next_beta, beta
        if (error < eps):
            break
    return beta

In [ ]:
from math import sin, sqrt

def f(x: float) -> float:
    return x**2 - 17.0 * sin(7.0 / 9.0 * x)

start, stop, extended = 0, 13, 3
x = np.linspace(start, stop, 2 ** 6)
extended_x = np.linspace(start - extended, stop + extended, 2 ** 20)
f_vectorize = np.vectorize(f)
y = f_vectorize(x)

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(16, 9), dpi=400)
subplot = fig.add_subplot(111, facecolor='#FFFFFF')
subplot.plot(x, y, color='red', lw=2, label='y')
plt.legend()
plt.show()

In [ ]:
from numpy.random import normal

y_noise = y + normal(loc=0, scale=1.0, size=len(y))
fig = plt.figure(figsize=(16, 9), dpi=400)
subplot = fig.add_subplot(111, facecolor='#FFFFFF')
subplot.plot(x, y_noise, 'Xr', label='y_noise')
subplot.plot(x, y, color='green', lw=2, label='y')
plt.legend()
plt.show()

In [ ]:
poly = ordinary_least_squares(x, y_noise, order=2, p=0, alpha=0.0)
polyval = np.polyval(poly, extended_x)

fig = plt.figure(figsize=(16, 9), dpi=400)
subplot = fig.add_subplot(111, facecolor='#FFFFFF')
subplot.plot(x, y_noise, 'Xr', label='y_noise')
subplot.plot(x, y, color='green', lw=2, label='y')
subplot.plot(extended_x, polyval, color='blue', lw=2, label='polyval')
plt.legend()
plt.show()
print(poly)

In [ ]:
poly = ordinary_least_squares(x, y_noise, order=4, p=0, alpha=0.0)
polyval = np.polyval(poly, x)

fig = plt.figure(figsize=(16, 9), dpi=400)
subplot = fig.add_subplot(111, facecolor='#FFFFFF')
subplot.plot(x, y_noise, 'Xr', label='y_noise')
subplot.plot(x, y, color='green', lw=2, label='y')
subplot.plot(x, polyval, color='blue', lw=2, label='polyval')
plt.legend()
plt.show()
print(poly)

In [ ]:
poly = ordinary_least_squares(x, y_noise, order=4, p=0, alpha=0.0)
polyval = np.polyval(poly, extended_x)

fig = plt.figure(figsize=(16, 9), dpi=400)
subplot = fig.add_subplot(111, facecolor='#FFFFFF')
subplot.plot(x, y_noise, 'Xr', label='y_noise')
subplot.plot(x, y, color='green', lw=2, label='y')
subplot.plot(extended_x, polyval, color='blue', lw=2, label='polyval')
plt.legend()
plt.show()
print(poly)

In [ ]:
colors = ['brown', 'maroon', 'darkgreen', 'blue', 'magenta', 'gold']
fig = plt.figure(figsize=(16, 9), dpi=400)
subplot = fig.add_subplot(111, facecolor='#FFFFFF')
subplot.plot(x, y_noise, 'Xr', label='y_noise')
subplot.plot(x, y, color='green', lw=2, label='y')

for i in range(6):
    alpha = 0.005 * (10 ** i)
    poly = ordinary_least_squares(x, y_noise, order=4, p=2, alpha=alpha)
    polyval = np.polyval(poly, extended_x)
    label = 'polyval: alpha = ' + str(alpha)
    subplot.plot(extended_x, polyval, color=colors[i], lw=2, label=label)
    print(poly)
plt.legend()
plt.show()